In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

# Определение формата пагинации
base_url = 'https://lifehacker.ru/topics/technology/'
pagination_format = '?page='  # Подтверждённый формат из результатов поиска


In [2]:
url = 'https://lifehacker.ru/topics/technology/'  # стартовая страница
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Поиск ссылок пагинации
pagination_links = soup.find_all('a', href=True)
for link in pagination_links:
    href = link['href']
    if 'page=' in href or '/page/' in href:
        print(href)

/topics/technology/?page=2
/topics/technology/?page=3
/topics/technology/?page=4
/topics/technology/?page=5
/topics/technology/?page=6
/topics/technology/?page=7
/topics/technology/?page=988
/topics/technology/?page=2
/topics/technology/?page=3
/topics/technology/?page=988
/topics/technology/?page=2


In [3]:
# Сбор ссылок на материалы
article_links = []
for page in tqdm(range(1, 11)):
    url = f"{base_url}{pagination_format}{page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Поиск ссылок на статьи (адаптировано под текущую структуру)
    links = soup.find_all('a', href=True)
    for link in links:
        href = link['href']
        if '/topics/technology/' in href and href != base_url:
            article_links.append(href)

    time.sleep(3)  # Задержка между запросами

# Удаление дубликатов
article_links = list(set(article_links))
# Печать всех ссылок на статьи
for link in article_links:
    print(link)

100%|██████████| 10/10 [00:48<00:00,  4.88s/it]

/topics/technology/?page=5
/topics/technology/?page=3
/topics/technology/?page=6
/topics/technology/?page=11
/topics/technology/ios/
/topics/technology/windows/
/topics/technology/
/topics/technology/?page=4
/topics/technology/?page=8
/topics/technology/browsers/
/topics/technology/?page=2
/topics/technology/?page=12
/topics/technology/gadgets/
/topics/technology/?page=9
/topics/technology/?page=7
/topics/technology/?page=10
/topics/technology/web-services/
/topics/technology/android/
/topics/technology/?page=988


In [4]:
from urllib.parse import urljoin

base_url = 'https://lifehacker.ru/'  # базовый URL сайта

data = []

for link in tqdm(article_links):
    try:
        full_url = urljoin(base_url, link)
        response = requests.get(full_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # Извлечение заголовка
            h1_tag = soup.find('h1')
            title = h1_tag.get_text(strip=True) if h1_tag else 'No title'
            # Извлечение содержания
            content = ' '.join([p.get_text(strip=True) for p in soup.find_all('p')])
            data.append({
                'title': title,
                'content': content,
                'url': full_url
            })
        else:
            print(f"Ошибка загрузки {full_url}: статус-код {response.status_code}")
        time.sleep(1)
    except requests.RequestException as e:
        print(f"Ошибка сети при обработке {full_url}: {e}")
    except Exception as e:
        print(f"Общая ошибка при обработке {full_url}: {e}")

100%|██████████| 19/19 [00:40<00:00,  2.11s/it]


In [5]:
# Создание DataFrame
df = pd.DataFrame(data)
print(df.head())

        title                                            content  \
0  Технологии  Новые комментарии 0/0 0/0 0/0 0/0 Весело и поз...   
1  Технологии  Новые комментарии 0/0 0/0 0/0 0/0 Бренд домашн...   
2  Технологии  Новые комментарии 0/0 0/0 0/0 0/0 Бренд домашн...   
3  Технологии  Новые комментарии 0/0 0/0 0/0 0/0 8 функций ум...   
4         iOS  Новые комментарии 0/0 0/0 0/0 0/0 Бренд домашн...   

                                                url  
0   https://lifehacker.ru/topics/technology/?page=5  
1   https://lifehacker.ru/topics/technology/?page=3  
2   https://lifehacker.ru/topics/technology/?page=6  
3  https://lifehacker.ru/topics/technology/?page=11  
4      https://lifehacker.ru/topics/technology/ios/  
